In [1]:
import os

In [2]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\Spam-email\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\N Ramaraju\\Desktop\\Projects\\Spam-email'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [12]:
from Spam_mail.utils.common import *
from Spam_mail.constants import *

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [17]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up

    def Data_transformation(self):
        # Read the data from CSV using the path from config
        df = pd.read_csv(self.config.data_path)
        logger.info('Reading Data...')
        logger.info('Vectorization initiated')
        # Vectorization using CountVectorizer
        cv = CountVectorizer()
        x = df['Message']
        y = df['Category'].apply(lambda x: 1 if x == 'spam' else 0)  # Convert labels to binary values
        x_vectorized = cv.fit_transform(x)
        logger.info('Vectorization Completed')


        # Save CountVectorizer to a file
        vectorizer_path = os.path.join(self.config.root_dir, 'count_vectorizer.pkl')
        joblib.dump(cv, vectorizer_path)

        # SMOTE oversampling
        smote = SMOTE(random_state=42)
        x_resampled, y_resampled = smote.fit_resample(x_vectorized, y)
        logger.info('Oversampling Completed')
        # Splitting Dataset into training and testing sets
        x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, test_size=0.2, random_state=42)
        logger.info('Data Splitting is Completed')
        return x_train, x_test, y_train, y_test


In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.Data_transformation()
except Exception as e:
    raise e

[2024-07-04 10:18:43,383: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-04 10:18:43,387: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-04 10:18:43,387: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-04 10:18:43,387: INFO: common: created directory at: artifacts]
[2024-07-04 10:18:43,393: INFO: common: created directory at: artifacts/data_transformation]
[2024-07-04 10:18:43,413: INFO: 1148571369: Reading Data...]
[2024-07-04 10:18:43,413: INFO: 1148571369: Vectorization initiated]
[2024-07-04 10:18:43,603: INFO: 1148571369: Vectorization Completed]
[2024-07-04 10:18:43,676: INFO: 1148571369: Oversampling Completed]
[2024-07-04 10:18:43,687: INFO: 1148571369: Data Splitting is Completed]
